In [ ]:
%pip install pandas
%pip install matplotlib
%pip install seaborn as sns
%pip install scikit-learn


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
import numpy as np

In [23]:
stock_file_path = './all_stock_and_etfs.csv'  

In [50]:
stock_df = pd.read_csv(stock_file_path)
stock_df

,Date-Time,Open,High,Low,Close,Volume,Ticker_Symbol
0,2024-02-20,29.09,29.7100,29.0001,29.44,2673889,HRL
1,2024-02-16,29.01,29.1700,28.7650,28.99,5655091,HRL
2,2024-02-15,28.94,29.2210,28.8500,29.11,2380372,HRL
3,2024-02-14,28.76,28.8254,28.5100,28.80,2690379,HRL
4,2024-02-13,29.56,29.8000,28.6350,28.72,4173650,HRL
...,...,...,...,...,...,...,...
163552,1999-11-05,9.25,9.5000,9.2500,9.50,2300,ALG
163553,1999-11-04,9.31,9.4400,9.3100,9.31,1100,ALG
163554,1999-11-03,9.00,9.2500,9.0000,9.19,35000,ALG
163555,1999-11-02,8.75,8.8800,8.6200,8.88,7700,ALG


## Generate Monthly ETF Values

In [51]:
stock_df['Date-Time'] = pd.to_datetime(stock_df['Date-Time'])

stock_df['Year'] = stock_df['Date-Time'].dt.year
stock_df['Month'] = stock_df['Date-Time'].dt.month

data_grouped = stock_df.groupby(['Year', 'Month', 'Ticker_Symbol']).agg({
    'Close': 'mean'
}).reset_index()

data_grouped.columns = ['Year', 'Month', 'Ticker_Symbol', 'Average_Close']
data_grouped

,Year,Month,Ticker_Symbol,Average_Close
0,1999,11,ADM,12.525238
1,1999,11,AGCO,11.762381
2,1999,11,ALG,9.661429
3,1999,11,CAG,24.602857
4,1999,11,CAT,52.900000
...,...,...,...,...
7809,2024,2,TSN,54.019231
7810,2024,2,VMI,231.409231
7811,2024,2,VOO,457.941250
7812,2024,2,WEN,18.983077


## VWAP 

In [52]:
stock_df['Date-Time'] = pd.to_datetime(stock_df['Date-Time'])
stock_df['Year'] = stock_df['Date-Time'].dt.year
stock_df['Month'] = stock_df['Date-Time'].dt.month

stock_df['Dollar_Volume'] = stock_df['Close'] * stock_df['Volume']

data_grouped = stock_df.groupby(['Year', 'Month', 'Ticker_Symbol']).agg({
    'Dollar_Volume': 'sum',
    'Volume': 'sum'
}).reset_index()

data_grouped['VWAP'] = data_grouped['Dollar_Volume'] / data_grouped['Volume']

data_grouped = data_grouped[['Year', 'Month', 'Ticker_Symbol', 'VWAP']]  #Volume-Weighted Average Price (VWAP)
data_grouped

,Year,Month,Ticker_Symbol,VWAP
0,1999,11,ADM,12.684922
1,1999,11,AGCO,11.911326
2,1999,11,ALG,9.658795
3,1999,11,CAG,24.660198
4,1999,11,CAT,51.796246
...,...,...,...,...
7809,2024,2,TSN,54.421946
7810,2024,2,VMI,231.210815
7811,2024,2,VOO,457.359215
7812,2024,2,WEN,18.909985


In [53]:
unique_symbols = data_grouped['Ticker_Symbol'].unique()
print("Unique Ticker Symbols:")
print(len(unique_symbols))

unique_symbols

Unique Ticker Symbols:
29


array(['ADM', 'AGCO', 'ALG', 'CAG', 'CAT', 'COKE', 'DE', 'DIA', 'DRI',
       'GIS', 'HRL', 'HSY', 'MCD', 'PEP', 'PPC', 'SAP', 'SBUX', 'SPY',
       'TSCO', 'TSN', 'VMI', 'WEN', 'YUM', 'ONEQ', 'DPZ', 'CMG', 'VOO',
       'CNHI', 'QSR'], dtype=object)

## Use Feature Engineered 5 year window index

In [54]:
data_grouped2 = data_grouped.copy()
data_grouped2['5Y_Avg_Close'] = data_grouped2.groupby('Ticker_Symbol')['VWAP'].transform(lambda x: x.rolling(window=60, min_periods=1).mean())
data_grouped2[data_grouped2['Year'] == 2010]

,Year,Month,Ticker_Symbol,VWAP,5Y_Avg_Close
2995,2010,1,ADM,30.541950,31.378890
2996,2010,1,AGCO,33.903972,33.145547
2997,2010,1,ALG,17.704802,19.712759
2998,2010,1,CAG,23.261748,22.475673
2999,2010,1,CAT,57.494634,63.952558
...,...,...,...,...,...
3306,2010,12,TSN,17.175906,15.097823
3307,2010,12,VMI,86.445921,70.277640
3308,2010,12,VOO,56.941744,54.216175
3309,2010,12,WEN,4.736514,9.728018


In [55]:
data_grouped2['Date'] = pd.to_datetime(data_grouped2[['Year', 'Month']].assign(day=1))

synthetic_etfs = {
    "Food_Beverage_ETF": ["HRL", "ADM", "PEP", "CAG", "HSY", "TSN", "GIS", "COKE", "PPC"],
    "Restaurant_FastFood_ETF": ["QSR", "DRI", "SBUX", "CMG", "WEN", "YUM", "MCD", "DPZ"],
    "Retail_ETF": ["TSCO"],
    "Agri_Machinery_ETF": ["AGCO", "CAT", "DE", "CNHI", "VMI", "ALG"],
    "Investment_Funds_ETF": ["VOO", "DIA", "ONEQ", "SPY"]
}

etf_prices = pd.DataFrame(columns=['Year', 'Month', 'ETF', 'Price', 'Volatility'])

for etf_name, stocks in synthetic_etfs.items():
    etf_data = data_grouped2[data_grouped2['Ticker_Symbol'].isin(stocks)].copy()
    
    etf_data['Weighted_Price'] = etf_data['VWAP'] * (etf_data['5Y_Avg_Close'] / etf_data.groupby(['Year', 'Month'])['5Y_Avg_Close'].transform('sum'))
    
    etf_grouped = etf_data.groupby(['Year', 'Month']).agg({
        'Weighted_Price': 'sum',
        'VWAP': 'std'  # Volatility
    }).reset_index()
    
    etf_grouped['ETF'] = etf_name
    etf_grouped.rename(columns={'Weighted_Price': 'Price', 'VWAP': 'Volatility'}, inplace=True)
    
    etf_prices = pd.concat([etf_prices, etf_grouped], ignore_index=True)

etf_prices.sort_values(by=['Year', 'Month', 'ETF'], inplace=True)

etf_prices



/var/folders/1x/443khrvj3wxbl0fy_v6jmhjh0000gn/T/ipykernel_550/4112817601.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  etf_prices = pd.concat([etf_prices, etf_grouped], ignore_index=True)


,Year,Month,ETF,Price,Volatility
876,1999,11,Agri_Machinery_ETF,36.835142,18.702643
0,1999,11,Food_Beverage_ETF,40.019708,16.961994
1168,1999,11,Investment_Funds_ETF,125.522357,22.170051
292,1999,11,Restaurant_FastFood_ETF,34.920701,12.464929
584,1999,11,Retail_ETF,17.868213,NaN
...,...,...,...,...,...
1167,2024,2,Agri_Machinery_ETF,277.317478,133.030308
291,2024,2,Food_Beverage_ETF,421.236543,269.684486
1459,2024,2,Investment_Funds_ETF,391.489413,197.982956
583,2024,2,Restaurant_FastFood_ETF,1607.725593,860.478030


In [56]:
etf_prices['Volatility'].fillna(0, inplace=True)


/var/folders/1x/443khrvj3wxbl0fy_v6jmhjh0000gn/T/ipykernel_550/3409463154.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  etf_prices['Volatility'].fillna(0, inplace=True)


In [57]:
etf_prices

,Year,Month,ETF,Price,Volatility
876,1999,11,Agri_Machinery_ETF,36.835142,18.702643
0,1999,11,Food_Beverage_ETF,40.019708,16.961994
1168,1999,11,Investment_Funds_ETF,125.522357,22.170051
292,1999,11,Restaurant_FastFood_ETF,34.920701,12.464929
584,1999,11,Retail_ETF,17.868213,0.000000
...,...,...,...,...,...
1167,2024,2,Agri_Machinery_ETF,277.317478,133.030308
291,2024,2,Food_Beverage_ETF,421.236543,269.684486
1459,2024,2,Investment_Funds_ETF,391.489413,197.982956
583,2024,2,Restaurant_FastFood_ETF,1607.725593,860.478030


In [46]:
etf_prices.to_csv('variables_of_interest/normalized_etf_monthly.csv')

## Less Elaborate Approach: Simply Add up the averages

In [29]:
import os 

synthetic_etfs = {
    "Food_Beverage_ETF": ["HRL", "ADM", "PEP", "CAG", "HSY", "TSN", "GIS", "COKE", "PPC"],
    "Restaurant_FastFood_ETF": ["QSR", "DRI", "SBUX", "CMG", "WEN", "YUM", "MCD", "DPZ"],
    "Retail_ETF": ["TSCO"],
    "Agri_Machinery_ETF": ["AGCO", "CAT", "DE", "CNHI", "VMI", "ALG"],
    "Investment_Funds_ETF": ["VOO", "DIA", "ONEQ", "SPY"]
}

etf_prices = pd.DataFrame(columns=['Year', 'Month', 'ETF', 'Price'])
for etf_name, stocks in synthetic_etfs.items():
    etf_data = data_grouped[data_grouped['Ticker_Symbol'].isin(stocks)]
    
    etf_grouped = etf_data.groupby(['Year', 'Month'])['Price'].sum().reset_index()
    
    etf_grouped['ETF'] = etf_name
    
    etf_grouped.rename(columns={'Price': 'Price'}, inplace=True)
    
    etf_prices = pd.concat([etf_prices, etf_grouped], ignore_index=True)

etf_prices

/var/folders/1x/443khrvj3wxbl0fy_v6jmhjh0000gn/T/ipykernel_550/277437212.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  etf_prices = pd.concat([etf_prices, etf_grouped], ignore_index=True)


,Year,Month,ETF,Price
0,1999,11,Food_Beverage_ETF,288.333177
1,1999,12,Food_Beverage_ETF,269.193649
2,2000,1,Food_Beverage_ETF,259.143417
3,2000,2,Food_Beverage_ETF,232.924240
4,2000,3,Food_Beverage_ETF,220.281292
...,...,...,...,...
1455,2023,10,Investment_Funds_ETF,1199.832527
1456,2023,11,Investment_Funds_ETF,1253.491332
1457,2023,12,Investment_Funds_ETF,1324.994446
1458,2024,1,Investment_Funds_ETF,1354.222311


In [30]:
etf_prices[etf_prices['Year'] == 1999]

,Year,Month,ETF,Price
0,1999,11,Food_Beverage_ETF,288.333177
1,1999,12,Food_Beverage_ETF,269.193649
292,1999,11,Restaurant_FastFood_ETF,154.488855
293,1999,12,Restaurant_FastFood_ETF,142.745241
584,1999,11,Retail_ETF,17.868213
585,1999,12,Retail_ETF,14.447012
876,1999,11,Agri_Machinery_ETF,130.615714
877,1999,12,Agri_Machinery_ETF,126.250292
1168,1999,11,Investment_Funds_ETF,247.065928
1169,1999,12,Investment_Funds_ETF,255.550197


In [8]:
etf_prices.to_csv('variables_of_interest/etf_monthly.csv')

## Generate Yearly ETF Data

In [58]:
yearly_df = etf_prices.groupby(['Year', 'ETF']).agg({'Price': 'mean', 'Volatility': 'mean'}).reset_index()

yearly_df

,Year,ETF,Price,Volatility
0,1999,Agri_Machinery_ETF,35.896562,18.041195
1,1999,Food_Beverage_ETF,38.574624,16.331450
2,1999,Investment_Funds_ETF,127.576142,21.784007
3,1999,Restaurant_FastFood_ETF,33.656642,12.122728
4,1999,Retail_ETF,16.157612,0.000000
...,...,...,...,...
125,2024,Agri_Machinery_ETF,275.591445,132.439912
126,2024,Food_Beverage_ETF,423.274203,272.971293
127,2024,Investment_Funds_ETF,383.809935,194.383929
128,2024,Restaurant_FastFood_ETF,1523.165088,812.518782


In [ ]:
yearly_df.to_csv('variables_of_interest/normalized_etf_yearly.csv')